In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#데이터 불러오기
df = pd.read_csv('./data/samsung.csv', encoding="cp949")

In [3]:
#데이터의 구성을 확인하기 
df.head()

,일자,시가,고가,저가,종가,거래량
0,2020-01-31,"57,800","58,400","56,400","56,400","19,749,457"
1,2020-01-30,"58,800","58,800","56,800","57,200","20,821,939"
2,2020-01-29,"59,100","59,700","58,800","59,100","16,446,102"
3,2020-01-28,"59,400","59,400","58,300","58,800","23,664,541"
4,2020-01-23,"61,800","61,800","60,700","60,800","14,916,555"


In [4]:
#데이터가 얼마나 있는지 확인하기
df.tail()

,일자,시가,고가,저가,종가,거래량
421,2018-05-11,"52,000","52,200","51,200","51,300","10,314,997"
422,2018-05-10,"51,700","51,700","50,600","51,600","13,905,263"
423,2018-05-09,"52,600","52,800","50,900","50,900","16,128,305"
424,2018-05-08,"52,600","53,200","51,900","52,600","23,104,720"
425,2018-05-04,"53,000","53,900","51,800","51,900","39,565,391"


In [5]:
#날짜를 기준으로 데이터 정렬해주기
df=df.sort_values(['일자'], ascending = [True])

In [6]:
#데이터 값들이 전부 문자열이므로 숫자들로 바꿔주기
for col in df.columns[1:]:  # Iterate over chosen columns
    df[col] = df[col].str.replace(",","")
    df[col] = pd.to_numeric(df[col])

In [7]:
#데이터 나누기
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, shuffle  = False) #train : 298행  / test : 128행

In [8]:
train, validation = train_test_split(train, test_size = 0.2, shuffle  = False)

In [9]:
#일자를 기준으로 순서 정렬
train = train.sort_values(['일자'], ascending = [True])
train.head()

,일자,시가,고가,저가,종가,거래량
425,2018-05-04,53000,53900,51800,51900,39565391
424,2018-05-08,52600,53200,51900,52600,23104720
423,2018-05-09,52600,52800,50900,50900,16128305
422,2018-05-10,51700,51700,50600,51600,13905263
421,2018-05-11,52000,52200,51200,51300,10314997


In [10]:
#'일자'열은 학습에 필요없으니 지워주기
del train['일자']

In [11]:
#'일자'가 사라졌는지 확인하기
train

,시가,고가,저가,종가,거래량
425,53000,53900,51800,51900,39565391
424,52600,53200,51900,52600,23104720
423,52600,52800,50900,50900,16128305
422,51700,51700,50600,51600,13905263
421,52000,52200,51200,51300,10314997
...,...,...,...,...,...
158,44800,45000,44550,44850,6664872
157,44800,45050,44300,44600,8607439
156,44200,44400,43400,43750,16906541
155,43750,44150,43300,44000,9322873


In [12]:
#train에서 학습용 y_train 분리
y_train = train['종가']

In [13]:
#train에서 학습용 y_train 분리한 다음 '종가'열 지워주기
del train['종가']

In [14]:
#train에서 종가열이 사라졌으니 x_train으로 이름 변경
x_train = train

In [15]:
#Validation 데이터도 마찬가지
y_val = validation['종가']

In [16]:
del validation['종가']

In [17]:
x_val = validation

In [18]:
del x_val['일자']

In [19]:
#일자를 기준으로 순서 정렬
test = test.sort_values(['일자'], ascending = [True])
test.head()

,일자,시가,고가,저가,종가,거래량
85,2019-09-25,49200,49350,48800,48900,9187141
84,2019-09-26,49000,49250,48900,49200,8494756
83,2019-09-27,48000,48700,48000,48400,8048041
82,2019-09-30,48050,49250,47900,49050,9497119
81,2019-10-01,48900,49100,48650,48850,6206035


In [20]:
y_test = test['종가']

In [21]:
del test['종가']

In [22]:
x_test = test

In [23]:
del x_test['일자']

In [24]:
#Keras 모듈 불러오기
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

In [25]:
#모델 구성하기
model = Sequential()#---------------------모델 방식

model.add(Dense(10, input_shape = (4,))) #-Hidden Layer 1
model.add(Dense(15))#---------------------Hidden Layer 2
model.add(Dense(8)) #---------------------Hidden Layer 3
model.add(Dense(5)) #---------------------Hidden Layer 4
model.add(Dense(3)) #---------------------Hidden Layer 5
model.add(Dense(1)) #---------------------Output Layer

model.summary() #-------------------------레이어 형태&파라미터

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 15)                165       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 128       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 18        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 4         
Total params: 410
Trainable params: 410
Non-trai

In [26]:
#학습하기
model.compile(loss = 'mse', optimizer='adam', metrics=['mse']) 
model.fit(x_train, y_train, epochs = 200, batch_size=3, validation_data=(x_val, y_val)) 

Instructions for updating:
Use tf.cast instead.
Train on 272 samples, validate on 68 samples
Epoch 1/200
272/272 [==============================] - 1s 3ms/step - loss: 193351189460.5367 - mse: 193351204864.0000 - val_loss: 858547355.5294 - val_mse: 858547392.0000
Epoch 2/200
272/272 [==============================] - 0s 718us/step - loss: 803010287.3676 - mse: 803010176.0000 - val_loss: 805793407.4265 - val_mse: 805793344.0000
Epoch 3/200
272/272 [==============================] - 0s 663us/step - loss: 3081615438.0441 - mse: 3081614848.0000 - val_loss: 290855484.5294 - val_mse: 290855456.0000
Epoch 4/200
272/272 [==============================] - 0s 717us/step - loss: 1072364492.6278 - mse: 1072364352.0000 - val_loss: 9299203681.8824 - val_mse: 9299205120.0000
Epoch 5/200
272/272 [==============================] - 0s 734us/step - loss: 3239925734.5147 - mse: 3239925760.0000 - val_loss: 466350241.4412 - val_mse: 466350272.0000
Epoch 6/200
272/272 [==============================] - 0s 76

272/272 [==============================] - 0s 767us/step - loss: 139544368.4845 - mse: 139544384.0000 - val_loss: 1406554.2470 - val_mse: 1406554.3750
Epoch 49/200
272/272 [==============================] - 0s 691us/step - loss: 2981733.3067 - mse: 2981733.0000 - val_loss: 145228273.6471 - val_mse: 145228272.0000
Epoch 50/200
272/272 [==============================] - 0s 791us/step - loss: 1074503834.3686 - mse: 1074503808.0000 - val_loss: 39336952.5294 - val_mse: 39336956.0000
Epoch 51/200
272/272 [==============================] - 0s 796us/step - loss: 89421987.9381 - mse: 89421992.0000 - val_loss: 141857856.0588 - val_mse: 141857856.0000
Epoch 52/200
272/272 [==============================] - 0s 686us/step - loss: 415559229.0332 - mse: 415559360.0000 - val_loss: 138194484.4118 - val_mse: 138194496.0000
Epoch 53/200
272/272 [==============================] - 0s 841us/step - loss: 94293760.4997 - mse: 94293752.0000 - val_loss: 148349133.4706 - val_mse: 148349136.0000
Epoch 54/200
272/

Epoch 98/200
272/272 [==============================] - 0s 740us/step - loss: 259546173.0299 - mse: 259546144.0000 - val_loss: 220560323.4706 - val_mse: 220560336.0000
Epoch 99/200
272/272 [==============================] - 0s 809us/step - loss: 12670510649.5303 - mse: 12670511104.0000 - val_loss: 210449286.3088 - val_mse: 210449280.0000
Epoch 100/200
272/272 [==============================] - 0s 724us/step - loss: 160218886.2169 - mse: 160218896.0000 - val_loss: 246898055.7206 - val_mse: 246898048.0000
Epoch 101/200
272/272 [==============================] - 0s 763us/step - loss: 1148049910.6029 - mse: 1148050048.0000 - val_loss: 29467468.6618 - val_mse: 29467472.0000
Epoch 102/200
272/272 [==============================] - 0s 826us/step - loss: 130067595.6558 - mse: 130067592.0000 - val_loss: 37144963.5257 - val_mse: 37144964.0000
Epoch 103/200
272/272 [==============================] - 0s 799us/step - loss: 178376785.7489 - mse: 178376784.0000 - val_loss: 250304961.8824 - val_mse: 2

Epoch 148/200
272/272 [==============================] - 0s 681us/step - loss: 33531466.6927 - mse: 33531468.0000 - val_loss: 3156177.9632 - val_mse: 3156178.0000
Epoch 149/200
272/272 [==============================] - 0s 671us/step - loss: 5008368.7984 - mse: 5008369.5000 - val_loss: 393477.9891 - val_mse: 393478.0000
Epoch 150/200
272/272 [==============================] - 0s 656us/step - loss: 1766697.6903 - mse: 1766698.1250 - val_loss: 787882.3470 - val_mse: 787882.3750
Epoch 151/200
272/272 [==============================] - 0s 645us/step - loss: 2472204.9641 - mse: 2472204.2500 - val_loss: 304775.8017 - val_mse: 304775.8125
Epoch 152/200
272/272 [==============================] - 0s 660us/step - loss: 326294.5907 - mse: 326294.6562 - val_loss: 129564.6403 - val_mse: 129564.6484
Epoch 153/200
272/272 [==============================] - 0s 654us/step - loss: 1698684.1092 - mse: 1698684.0000 - val_loss: 110126.3636 - val_mse: 110126.3672
Epoch 154/200
272/272 [=====================

272/272 [==============================] - 0s 839us/step - loss: 2702492.3253 - mse: 2702491.5000 - val_loss: 860094.6185 - val_mse: 860094.6875
Epoch 200/200
272/272 [==============================] - 0s 805us/step - loss: 2260270.2749 - mse: 2260270.0000 - val_loss: 99124.1763 - val_mse: 99124.1797


In [31]:
#평가하기
loss, mse = model.evaluate(x_test,y_test,batch_size=1)
print('mse:', mse)
print('loss:',loss)

86/86 [==============================] - 0s 1ms/step
mse: 197428.109375
loss: 197428.14584581243


In [28]:
#예측하기
x_prd = np.array([[57100, 58800, 56800, 23995260]])
test = model.predict(x_prd, batch_size=4)
print(test)

[[57411.72]]


In [29]:
y_prd = model.predict(x_test, batch_size = 1) # <------- 이거 중요~! x_test와 y_test 매치가 중요하다

In [30]:
#RMSE 구하기
from sklearn.metrics import mean_squared_error

def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict)) 

print('RMSE : ', RMSE(y_test, y_prd))

RMSE :  444.3288706015061
